In [ ]:
# get yolov3 pretrained coco dataset weights
!wget https://pjreddie.com/media/files/yolov3.weights

--2023-06-12 13:24:25--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights.1’

yolov3.weights.1    100%[===================>] 236.52M  77.4MB/s    in 3.2s    

2023-06-12 13:24:29 (73.4 MB/s) - ‘yolov3.weights.1’ saved [248007048/248007048]



In [1]:
import cv2
import numpy as np
import pandas as pd
import math
import matplotlib.pyplot as plt

from keras.preprocessing import image # Preprocessing the images
from keras.utils import np_utils
from skimage.transform import resize


In [15]:
def Frame_by_Frame(videoFile):
  count=0
  cap=cv2.VideoCapture(videoFile) # capturing video
  frameRate=cap.get(5) #framerate
  if(not cap.isOpened()):
    print("Cannot open camera")
    exit()
  while True:
    frameId = cap.get(1) # current frame number
    ret,frame = cap.read() # capture frame by frame,it will return true if frame is read correctly,this will determine the end of the video
    if(ret!=True):
      break
    if(frameId % math.floor(frameRate)==0):
      filename="frame%d.jpg" % count;count+=1
      cv2.imwrite(filename,frame)
  cap.release()
  print("Done!")    



In [16]:
videoFile="/content/road_trafifc.mp4"

Frame_by_Frame(videoFile)


Done!


In [17]:
n=cv2.dnn.readNet("/content/yolov3.weights","/content/yolov3.cfg")
list_of_labels=[]
with open("/content/coco.names","r") as f:

  list_of_labels=[line.strip() for line in f.readlines()]
layers=n.getLayerNames()
out_layers = [layers[i-1] for i in n.getUnconnectedOutLayers()]
colors=np.random.uniform(0,255,size=(len(list_of_labels),3))


In [18]:
def Obj_detect(image_name):
    cur_img= cv2.imread(image_name)
    height, width, channels = cur_img.shape
    blob = cv2.dnn.blobFromImage(cur_img, 0.00392, (416,416), (0, 0, 0), True, crop=False)
    n.setInput(blob)
    outs = n.forward(out_layers)
    class_ids = []
    confidences = []
    boxes = []
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                # Object detected
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    count=0
    font = cv2.FONT_HERSHEY_PLAIN
    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            label = str(list_of_labels[class_ids[i]])
            if(label in ['car','bus','truck','motorbike','bicycle']):
                
                count+=1
            color = colors[i]
            cv2.rectangle(cur_img, (x, y), (x + w, y + h), color, 2)
            cv2.putText(cur_img, label, (x, y + 30), font, 3, color, 3)
    return count

In [ ]:
total_chaos=0
no_of_frames=0
for i in range(0,11):
  image_n="/content/frame"+str(i)+".jpg"
  s= Obj_detect(image_n)
  if(s!=0):
    total_chaos+=s
    no_of_frames+=1
  

In [ ]:
print(" Total Chaos  : ",total_chaos)
print("Useful no of frames : ",no_of_frames)

In [ ]:
avg_chaos=(total_chaos/no_of_frames)
print("Average Chaos : ",avg_chaos)